# Summarizer
Objective here is to produce a summary given a list of documents (texts)
- We are dealing with short documents (headlines, tweets) so we could either use single document summary by treating each headline as a sentence in a large document to summarize or we can use multidocument summary where each document is just the headline. Note that single document summarizers will likely use the position of the sentence in the document as a summary

Parts of the study
- We can use some evaluation too when applying across documents

# Quick start
Easiest approach is to use sentence transformers to embed each headline from a list and find the most similar within the cluster as representative

In [2]:
from sentence_transformers import SentenceTransformer
model1 = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [147]:
import pandas as pd
df = pd.read_csv('../et/data/abcnews-date-text.csv',index_col=0).sample(10000,random_state=10)

/Users/md/opt/anaconda3/envs/hac_viz/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [148]:
clust_txt=list(df.headline_text.values)

In [149]:
%%time
clust_emb=model1.encode(clust_txt, batch_size=16, 
                                       show_progress_bar=False, convert_to_numpy=True)

CPU times: user 4min 16s, sys: 2.05 s, total: 4min 18s
Wall time: 43.4 s


In [6]:
%load_ext autoreload

In [7]:
%autoreload 2
from picture_text_summary import unroll_tree_map, cluster_summary_simple
from hac_tools import HAC
from treemap import build_tree_map

In [ ]:
single
complete
average
weighted
Ward
centroid
median

- per documentation this scales O(n^2) lets see this with actual numbers and examples
- try out different clustering approaches and time taken
- the issue with hac is it has too much freedom - that is both its strength but the difficulty with visualising it

Experiments
- test different clusterings and the internal grouping average
- 

In [164]:
%%time
df_res = unroll_tree_map(clust_emb, method='ward',depth=6,min_size=0.05,max_extension=3,)
df_res.shape

CPU times: user 52.1 s, sys: 275 ms, total: 52.3 s
Wall time: 52.4 s


(1098, 5)

In [165]:
%%time
import numpy as np
df_res['labels'], df_res['color']= zip(*df_res.apply(lambda x: \
    cluster_summary_simple([np.array(clust_txt[m]) for m in x['cluster_members']], \
                           model1, [np.array(clust_emb[m]) for m in x['cluster_members']]), axis=1))
average_score = df_res.apply(lambda x: x['color']*x['value'],axis=1).sum()/df_res.value.sum()
print(df_res.shape, average_score)


(1098, 7) 0.7200893283933488
CPU times: user 15.5 s, sys: 2.42 s, total: 17.9 s
Wall time: 3.37 s


In [166]:
#build_tree_map(df_res1.iloc[::-1])

build_tree_map(df_res,maxdepth=3,average_score=0.7)

